In [1]:
%%time
%%capture
!pip install peft
!pip install evaluate
!pip install datasets

CPU times: user 253 ms, sys: 86 ms, total: 339 ms
Wall time: 1min 17s


In [2]:
%%time
import os
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from peft import PeftModel, LoraConfig, get_peft_model
import evaluate
import torch
import numpy as np
from sklearn.model_selection import train_test_split

2025-11-27 08:21:04.711322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764231664.881041      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764231664.928677      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

CPU times: user 18.9 s, sys: 3.96 s, total: 22.8 s
Wall time: 32.5 s


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [4]:
%%time
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
base_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 5.71 s, sys: 4.1 s, total: 9.81 s
Wall time: 6.14 s


In [5]:
TRAIN_DIR = "/kaggle/input/polar-dataset/subtask1/train"
DEV_DIR = "/kaggle/input/polar-dataset/subtask1/dev"

def load_split(split_dir):
    dfs = []
    for file in os.listdir(split_dir):
        if file.endswith(".csv"):
            lang = file.replace(".csv","")
            df = pd.read_csv(os.path.join(split_dir, file))
            df["lang"] = lang
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

main_df = load_split(TRAIN_DIR)
dev_df = load_split(DEV_DIR)

print("Train size:", main_df.shape)
print("Dev size:", dev_df.shape)

Train size: (40395, 4)
Dev size: (2012, 4)


In [6]:
# ===== BASIC EDA CHECKS =====

print("=== Dataset Shapes ===")
print("Train size:", main_df.shape)
print("Dev size:", dev_df.shape)
print()

# --- Duplicate checks ---
print("=== Duplicate Checks (TRAIN) ===")
print("Duplicate IDs:", main_df['id'].duplicated().sum())
print("Duplicate texts:", main_df['text'].duplicated().sum())
print()

# --- Missing values ---
print("=== Missing Values (TRAIN) ===")
print(main_df.isna().sum())
print()

# --- Polarization distribution ---
print("=== Polarization Distribution (TRAIN) ===")
print(main_df['polarization'].value_counts())
print()

print("Polarization distribution (%):")
print(main_df['polarization'].value_counts(normalize=True) * 100)
print()

# --- Language distribution ---
print("=== Language Distribution (TRAIN) ===")
print(main_df['lang'].value_counts())
print()

# print("Language distribution (%):")
# print(main_df['lang'].value_counts(normalize=True) * 100)
# print()

# --- Text length stats ---
# main_df["text_len"] = main_df["text"].astype(str).apply(len)

# print("=== Text Length Stats (TRAIN) ===")
# print(main_df["text_len"].describe())
# print()

# # --- Quick dev set distribution checks for reference ---
# print("=== Dev Set Polarization Distribution ===")
# print(dev_df['polarization'].value_counts())
# print()

# print("=== Dev Set Language Distribution ===")
# print(dev_df['lang'].value_counts())

=== Dataset Shapes ===
Train size: (40395, 4)
Dev size: (2012, 4)

=== Duplicate Checks (TRAIN) ===
Duplicate IDs: 0
Duplicate texts: 3

=== Missing Values (TRAIN) ===
id              0
text            0
polarization    0
lang            0
dtype: int64

=== Polarization Distribution (TRAIN) ===
polarization
1    21010
0    19385
Name: count, dtype: int64

Polarization distribution (%):
polarization
1    52.011388
0    47.988612
Name: proportion, dtype: float64

=== Language Distribution (TRAIN) ===
lang
zho    4280
hau    3651
arb    3380
ita    3334
amh    3332
spa    3305
fas    3295
deu    3180
urd    2849
hin    2744
eng    2676
tur    2364
nep    2005
Name: count, dtype: int64



In [7]:
# ---- STRATIFIED SPLIT 90 / 5 / 5 ----
train_df, temp_df = train_test_split(
    main_df,
    test_size=0.10,
    stratify=main_df["polarization"],
    random_state=42,
    shuffle=True
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,
    stratify=temp_df["polarization"],
    random_state=42,
    shuffle=True
)

# ---- KEEP ONLY REQUIRED COLUMNS ----
cols = ["text", "polarization"]

train_df = train_df[cols].rename(columns={"polarization": "labels"}).reset_index(drop=True)
val_df   = val_df[cols].rename(columns={"polarization": "labels"}).reset_index(drop=True)
test_df  = test_df[cols].rename(columns={"polarization": "labels"}).reset_index(drop=True)

# ---- PRINT SHAPES AND LABEL DISTRIBUTIONS ----
print("Train:", train_df.shape)
print("Val:", val_df.shape)
print("Test:", test_df.shape)

print("\nTrain distribution:\n", train_df["labels"].value_counts(normalize=True)*100)
print("\nVal distribution:\n", val_df["labels"].value_counts(normalize=True)*100)
print("\nTest distribution:\n", test_df["labels"].value_counts(normalize=True)*100)

Train: (36355, 2)
Val: (2020, 2)
Test: (2020, 2)

Train distribution:
 labels
1    52.012103
0    47.987897
Name: proportion, dtype: float64

Val distribution:
 labels
1    52.029703
0    47.970297
Name: proportion, dtype: float64

Test distribution:
 labels
1    51.980198
0    48.019802
Name: proportion, dtype: float64


In [8]:
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
val_dataset = Dataset.from_pandas(val_df[['text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['text', 'labels']])

In [9]:
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 36355
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 2020
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 2020
    })
})

In [10]:
%%time
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# Encode the input data
dataset = dataset.map(tokenize_function, batched=True)
# The transformers model expects the target class column to be named "labels"
# dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/36355 [00:00<?, ? examples/s]

Map:   0%|          | 0/2020 [00:00<?, ? examples/s]

Map:   0%|          | 0/2020 [00:00<?, ? examples/s]

CPU times: user 13.3 s, sys: 141 ms, total: 13.4 s
Wall time: 6.64 s


In [11]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

In [12]:
model = get_peft_model(base_model, lora_config)

In [13]:
%%capture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [14]:
training_args = TrainingArguments(
    output_dir="./output_results",
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
#     warmup_steps=500,
    logging_dir="./logs",
    logging_steps=500,
    learning_rate=2e-5,
    # weight_decay=0.01,
    optim="adamw_torch",
    gradient_accumulation_steps=2,
    load_best_model_at_end=True, # needed for early stopping
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    report_to="none",
)

metric = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Early stopping after 2 evaluations with no improvement
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.1)]
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
%%time
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.668600,0.571737,0.720297
1000,0.568000,0.539691,0.741584
1500,0.546000,0.526693,0.752970
2000,0.543900,0.516415,0.753465
2500,0.524100,0.515795,0.754950
3000,0.527800,0.510567,0.762871
3500,0.522700,0.501365,0.762871
4000,0.517600,0.497614,0.765842
4500,0.517000,0.499823,0.765347
5000,0.506500,0.491289,0.762871


CPU times: user 1h 25min 47s, sys: 6.26 s, total: 1h 25min 54s
Wall time: 1h 25min 52s


TrainOutput(global_step=7500, training_loss=0.5285695190429688, metrics={'train_runtime': 5152.3564, 'train_samples_per_second': 211.68, 'train_steps_per_second': 6.62, 'total_flos': 3.1850038684790784e+16, 'train_loss': 0.5285695190429688, 'epoch': 6.596568411790585})

In [16]:
%%time
trainer.evaluate()

CPU times: user 14.9 s, sys: 47 ms, total: 14.9 s
Wall time: 14.9 s


{'eval_loss': 0.4824202358722687,
 'eval_accuracy': 0.7698019801980198,
 'eval_runtime': 14.8749,
 'eval_samples_per_second': 135.799,
 'eval_steps_per_second': 8.538,
 'epoch': 6.596568411790585}

In [17]:
%%time
from sklearn.metrics import classification_report
# eval now
predictions = trainer.predict(dataset['test'])

# Extract predicted labels and true labels
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Compute metrics using sklearn's classification report
report = classification_report(labels, preds, target_names=['Not Polar (0)', 'Polar (1)'], digits=4)

print(report)

from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(labels, preds)
f1_macro = f1_score(labels, preds, average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1 Score: {f1_macro:.4f}")

               precision    recall  f1-score   support

Not Polar (0)     0.7572    0.8134    0.7843       970
    Polar (1)     0.8149    0.7590    0.7860      1050

     accuracy                         0.7851      2020
    macro avg     0.7861    0.7862    0.7851      2020
 weighted avg     0.7872    0.7851    0.7852      2020

Accuracy: 0.7851
Macro F1 Score: 0.7851
CPU times: user 14.8 s, sys: 44 ms, total: 14.9 s
Wall time: 14.9 s


In [18]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

accuracy = accuracy_score(labels, preds)

f1_macro = f1_score(labels, preds, average='macro')
f1_micro = f1_score(labels, preds, average='micro')
f1_weighted = f1_score(labels, preds, average='weighted')

precision = precision_score(labels, preds, average='macro')
recall = recall_score(labels, preds, average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1 Score: {f1_macro:.4f}")
print(f"Micro F1 Score: {f1_micro:.4f}")
print(f"Weighted F1 Score: {f1_weighted:.4f}")

print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")

Accuracy: 0.7851
Macro F1 Score: 0.7851
Micro F1 Score: 0.7851
Weighted F1 Score: 0.7852
Precision (Macro): 0.7861
Recall (Macro): 0.7862


In [21]:
# df_results = pd.DataFrame({
#     'text': dataset["test"]['text'],
#     'true_label': dataset["test"]['labels'],
#     'predicted_label': np.argmax(predictions.predictions, axis=1)
# })

# df_results.to_csv('test_predictions.csv', index=False)

### script for submission

In [22]:
def predict_df(df, tokenizer, model):
    dataset = Dataset.from_pandas(df[["text"]])
    dataset = dataset.map(lambda x: tokenizer(
        x["text"], truncation=True, padding="max_length", max_length=256
    ))
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

    preds = trainer.predict(dataset).predictions
    pred_labels = np.argmax(preds, axis=1)

    return pred_labels

In [23]:
%%time
import os

OUTPUT_DIR = "subtask_1"
os.makedirs(OUTPUT_DIR, exist_ok=True)

languages = sorted(dev_df["lang"].unique())
print("Languages:", languages)

for lang in languages:
    lang_df = dev_df[dev_df["lang"] == lang].reset_index(drop=True)
    
    # Run predictions
    pred_labels = predict_df(lang_df, tokenizer, model)
    
    # Build Codabench format
    submission_df = pd.DataFrame({
        "id": lang_df["id"],
        "polarization": pred_labels
    })
    
    # Save file
    fname = f"pred_{lang}.csv"
    fpath = os.path.join(OUTPUT_DIR, fname)
    submission_df.to_csv(fpath, index=False)
    
    print(f"Saved: {fpath}")

Languages: ['amh', 'arb', 'deu', 'eng', 'fas', 'hau', 'hin', 'ita', 'nep', 'spa', 'tur', 'urd', 'zho']


Map:   0%|          | 0/166 [00:00<?, ? examples/s]

Saved: subtask_1/pred_amh.csv


Map:   0%|          | 0/169 [00:00<?, ? examples/s]

Saved: subtask_1/pred_arb.csv


...

Saved: subtask_1/pred_tur.csv


Map:   0%|          | 0/142 [00:00<?, ? examples/s]

Saved: subtask_1/pred_urd.csv


Map:   0%|          | 0/214 [00:00<?, ? examples/s]

Saved: subtask_1/pred_zho.csv
CPU times: user 16.3 s, sys: 90.1 ms, total: 16.4 s
Wall time: 16.3 s


In [24]:
!zip -r subtask_1.zip subtask_1

  adding: subtask_1/ (stored 0%)
  adding: subtask_1/pred_urd.csv (deflated 48%)
  adding: subtask_1/pred_eng.csv (deflated 48%)
  adding: subtask_1/pred_zho.csv (deflated 48%)
  adding: subtask_1/pred_hin.csv (deflated 48%)
  adding: subtask_1/pred_tur.csv (deflated 48%)
  adding: subtask_1/pred_amh.csv (deflated 48%)
  adding: subtask_1/pred_deu.csv (deflated 48%)
  adding: subtask_1/pred_fas.csv (deflated 48%)
  adding: subtask_1/pred_ita.csv (deflated 48%)
  adding: subtask_1/pred_spa.csv (deflated 48%)
  adding: subtask_1/pred_arb.csv (deflated 48%)
  adding: subtask_1/pred_hau.csv (deflated 48%)
  adding: subtask_1/pred_nep.csv (deflated 48%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from IPython.display import FileLink
display(FileLink('/kaggle/working/subtask_1.zip'))

/kaggle/working/subtask_1.zip